<a href="https://colab.research.google.com/github/deep1003/deep1003/blob/master/PatentSBERTa_0901.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 목적: 이 코드의 목적은 AI 특허와 비-AI 특허를 분류하는 것입니다.
# Sentence BERT 기반의 모델을 사용하여 특허 문서의 임베딩을 생성하고,
# FAISS를 이용한 Approximate Nearest Neighbors (ANN) 알고리즘을 활용하여
# 각 특허가 AI 관련 특허인지 아닌지를 판별합니다.

# 1. 필수 라이브러리 설치 및 데이터 로드
# sentence-transformers: BERT 기반 임베딩 모델을 불러오기 위한 라이브러리
# faiss-gpu: FAISS 라이브러리로, 대규모 데이터에서 유사도 검색을 빠르게 수행하는 데 사용
!pip install -q faiss-gpu sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

# 데이터셋 로드: AI로 분류된 특허 데이터와 전체 특허 데이터 불러오기
df_subset_AI = pd.read_csv('df_subset_AI.csv')  # AI로 분류된 특허 데이터
df_fullset_patents = pd.read_csv('df_fullset_patents.csv')  # 전체 특허 데이터


In [ ]:

# 2. PatentSBERTa를 사용하여 특허 임베딩 생성
# PatentSBERTa 모델 로드: AI 특허 분류를 위한 임베딩을 생성하기 위해 특화된 BERT 모델
model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

# AI로 분류된 특허의 임베딩 생성
# AI 특허 문서 텍스트를 임베딩 벡터로 변환
ai_embeddings = model.encode(df_subset_AI['text_column'].tolist(), convert_to_tensor=True)

# 전체 특허 데이터의 임베딩 생성
# 전체 특허 문서 텍스트를 임베딩 벡터로 변환
fullset_embeddings = model.encode(df_fullset_patents['text_column'].tolist(), convert_to_tensor=True)

In [ ]:


# 3. FAISS를 이용한 Approximate Nearest Neighbors (ANN) 구현
# FAISS 라이브러리를 사용해 빠른 유사도 검색을 수행합니다.
# 이를 통해 대규모 데이터셋에서 AI 특허와 유사한 문서를 검색할 수 있습니다.

# 임베딩을 FAISS에서 사용하기 위해 numpy 배열로 변환
ai_embeddings_np = ai_embeddings.cpu().numpy()
fullset_embeddings_np = fullset_embeddings.cpu().numpy()

# FAISS 인덱스 초기화: 임베딩의 차원 수를 기반으로 L2 거리 계산을 위한 인덱스 생성
d = ai_embeddings_np.shape[1]  # 임베딩의 차원 수
index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성

# 인덱스에 AI로 분류된 특허 임베딩 추가
index.add(ai_embeddings_np)

# 인덱스를 사용해 전체 특허 데이터에서 가장 유사한 k개의 이웃을 검색
k = 5  # 검색할 가장 가까운 이웃의 수
D, I = index.search(fullset_embeddings_np, k)


In [ ]:

# 4. 유사도를 기반으로 특허 분류
# 검색된 이웃들 중 유사도 점수와 기준을 통해 AI 특허인지 여부를 판단합니다.
# 임계값을 설정하거나 KNN을 사용하여 특허를 AI와 비-AI로 분류할 수 있습니다.

# AI 분류 결과를 저장할 리스트 생성
ai_classification = []

# 유사도 점수를 기반으로 AI 여부를 분류
threshold = 0.75  # AI 특허로 분류하기 위한 유사도 임계값

for i in range(len(D)):
    # k개의 이웃들에 대한 평균 유사도 계산
    avg_distance = D[i].mean()

    # 평균 유사도가 임계값보다 낮으면 AI로 분류
    if avg_distance < threshold:
        ai_classification.append('AI')
    else:
        ai_classification.append('Non-AI')

# 분류 결과를 DataFrame에 추가
df_fullset_patents['AI_Class'] = ai_classification


In [ ]:

# 5. 결과 저장
# 최종 분류 결과를 CSV 파일로 저장하여 분석에 사용
df_fullset_patents.to_csv('classified_patents_with_ann.csv', index=False)


In [ ]:

# 6. 모델 평가 (선택 사항)
# 실제 레이블과 비교하여 모델 성능을 평가할 수 있습니다.
from sklearn.metrics import classification_report

# 실제 레이블과 분류된 레이블 비교 후 성능 평가 (F1 스코어, 정밀도, 재현율 등)
true_labels = df_fullset_patents['true_label']  # 실제 레이블이 있는 경우
print(classification_report(true_labels, df_fullset_patents['AI_Class']))
